
## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [1]:
%matplotlib inline
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
import xml.etree.ElementTree as et

In [2]:
labelencoder = LabelEncoder()

## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [3]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

In [4]:
df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
print("The shape of the dataset is {}.\n\n".format(df.shape))
df.head()

The shape of the dataset is (6407, 16).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [5]:
df.drop(columns='ID').describe()

,Lat,Lng,Distance(mi),Severity
count,6407.000000,6407.000000,6407.000000,6407.000000
mean,37.765653,-122.405990,0.135189,2.293429
std,0.032555,0.028275,0.396360,0.521225
min,37.609619,-122.510440,0.000000,1.000000
25%,37.737096,-122.412210,0.000000,2.000000
50%,37.768238,-122.404835,0.000000,2.000000
75%,37.787813,-122.392477,0.041000,3.000000
max,37.825626,-122.349734,6.820000,4.000000


The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

### Remove columns with now effect

In [6]:
df.drop(columns=['Bump','Roundabout'],inplace=True)

#### Format date data in the dataframe and extract the hour to match with weather data

In [7]:
df['timestamp']=pd.to_datetime(df['timestamp'])
df['date_time']=pd.to_datetime(df['timestamp'].dt.strftime('%Y-%m-%d %H:00:00'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6407 entries, 0 to 6406
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            6407 non-null   int64         
 1   Lat           6407 non-null   float64       
 2   Lng           6407 non-null   float64       
 3   Distance(mi)  6407 non-null   float64       
 4   Crossing      6407 non-null   bool          
 5   Give_Way      6407 non-null   bool          
 6   Junction      6407 non-null   bool          
 7   No_Exit       6407 non-null   bool          
 8   Railway       6407 non-null   bool          
 9   Stop          6407 non-null   bool          
 10  Amenity       6407 non-null   bool          
 11  Side          6407 non-null   object        
 12  Severity      6407 non-null   int64         
 13  timestamp     6407 non-null   datetime64[ns]
 14  date_time     6407 non-null   datetime64[ns]
dtypes: bool(7), datetime64[ns](2), float64

#### convert bool values to numeric int values for cleaning

In [8]:
df['Crossing'] = df['Crossing'].astype('int')
df['Give_Way'] = df['Give_Way'].astype('int')
df['Junction'] = df['Junction'].astype('int')
df['No_Exit'] = df['No_Exit'].astype('int')
df['Railway'] = df['Railway'].astype('int')
df['Stop'] = df['Stop'].astype('int')
df['Amenity'] = df['Amenity'].astype('int')

#### encode side data as numeric

In [9]:
df['Side'] = labelencoder.fit_transform(df['Side'])

In [10]:
df.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp,date_time
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,1,2,2016-03-25 15:13:02,2016-03-25 15:00:00
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,1,2,2020-05-05 19:23:00,2020-05-05 19:00:00
2,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,1,3,2016-09-16 19:57:16,2016-09-16 19:00:00
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,1,1,2020-03-29 19:48:43,2020-03-29 19:00:00
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,1,2,2019-10-09 08:47:00,2019-10-09 08:00:00


### Import Holiday Data

In [11]:
xtree = et.parse(os.path.join(dataset_path, 'holidays.xml'))
xroot = xtree.getroot()
df_cols = ["date", "description"]
rows = []
for node in xroot:
    date = node.find("date").text if node is not None else None
    description = node.find("description").text if node is not None else None
    rows.append({"date": date,"description": description})
holidays_df = pd.DataFrame(rows)
holidays_df.head()

,date,description
0,2012-01-02,New Year Day
1,2012-01-16,Martin Luther King Jr. Day
2,2012-02-20,Presidents Day (Washingtons Birthday)
3,2012-05-28,Memorial Day
4,2012-07-04,Independence Day


#### format date to match with the train data

In [12]:
holidays_df['date']=pd.to_datetime(holidays_df['date'])

#### Check if the day of the accident was holiday or not

In [13]:
df['is_holiday']=df['timestamp'].dt.date.isin(holidays_df['date'].dt.date).astype(int)  

### Import Weather Data

In [14]:
weather_df = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'))
weather_df['date_time']=pd.to_datetime(weather_df[["Year", "Month", "Day","Hour"]])
weather_df.dropna(subset=["Weather_Condition"],inplace=True)

In [15]:
weather_df['Weather_Condition'] = labelencoder.fit_transform(weather_df['Weather_Condition'])

In [16]:
weather_df['Visibility(mi)'].fillna(weather_df['Visibility(mi)'].mean(),inplace=True)
weather_df['Wind_Speed(mph)'].fillna(weather_df['Wind_Speed(mph)'].mean(),inplace=True)
weather_df['Humidity(%)'].fillna(weather_df['Humidity(%)'].mean(),inplace=True)
weather_df['Temperature(F)'].fillna(weather_df['Temperature(F)'].mean(),inplace=True)

In [17]:
weather_df.drop_duplicates(subset=['date_time'],inplace=True)

In [18]:
weather_df.drop(columns=['Wind_Chill(F)','Precipitation(in)','Selected',
                         "Year", "Month", "Day","Hour"],inplace=True)

In [19]:
weather_df.head()

,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),date_time
0,3,64.0,70.0,20.0,10.0,2020-07-27 18:00:00
1,17,71.1,57.0,9.2,10.0,2017-09-30 17:00:00
2,16,57.9,87.0,15.0,9.0,2017-06-27 05:00:00
3,0,66.9,73.0,4.6,10.0,2016-09-07 09:00:00
4,3,52.0,89.0,0.0,9.0,2019-10-19 02:00:00


In [20]:
df_merged =pd.merge(df, weather_df, how='left', on=['date_time'])

In [21]:
df_merged.dropna(inplace=True)

In [22]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6406 entries, 0 to 6406
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ID                 6406 non-null   int64         
 1   Lat                6406 non-null   float64       
 2   Lng                6406 non-null   float64       
 3   Distance(mi)       6406 non-null   float64       
 4   Crossing           6406 non-null   int64         
 5   Give_Way           6406 non-null   int64         
 6   Junction           6406 non-null   int64         
 7   No_Exit            6406 non-null   int64         
 8   Railway            6406 non-null   int64         
 9   Stop               6406 non-null   int64         
 10  Amenity            6406 non-null   int64         
 11  Side               6406 non-null   int64         
 12  Severity           6406 non-null   int64         
 13  timestamp          6406 non-null   datetime64[ns]
 14  date_tim

In [23]:
df_merged.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,...,Side,Severity,timestamp,date_time,is_holiday,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,0,37.762150,-122.405660,0.044,0,0,0,0,0,0,...,1,2,2016-03-25 15:13:02,2016-03-25 15:00:00,0,22.0,64.0,58.0,23.0,10.0
1,1,37.719157,-122.448254,0.000,0,0,0,0,0,0,...,1,2,2020-05-05 19:23:00,2020-05-05 19:00:00,0,15.0,57.0,83.0,22.0,10.0
2,2,37.808498,-122.366852,0.000,0,0,0,0,0,1,...,1,3,2016-09-16 19:57:16,2016-09-16 19:00:00,0,0.0,62.1,80.0,9.2,10.0
3,3,37.785930,-122.391080,0.009,0,0,1,0,0,0,...,1,1,2020-03-29 19:48:43,2020-03-29 19:00:00,0,3.0,58.0,70.0,10.0,10.0
4,4,37.719141,-122.448457,0.000,0,0,0,0,0,0,...,1,2,2019-10-09 08:47:00,2019-10-09 08:00:00,0,3.0,58.0,65.0,3.0,10.0


In [24]:
corr_matrix = df_merged.drop(columns=['ID']).corr()

In [25]:
corr_matrix["Severity"].sort_values(ascending=False)

Severity             1.000000
Stop                 0.229373
Lng                  0.145331
Weather_Condition    0.104449
Lat                  0.099815
Humidity(%)          0.076742
Side                 0.060505
Wind_Speed(mph)      0.021897
is_holiday           0.003168
No_Exit             -0.007033
Give_Way            -0.012183
Distance(mi)        -0.013071
Visibility(mi)      -0.015716
Temperature(F)      -0.018306
Railway             -0.033292
Junction            -0.068724
Amenity             -0.078885
Crossing            -0.090264
Name: Severity, dtype: float64

In [26]:
df_merged.drop(columns=['date_time','timestamp'],inplace=True)

In [27]:
df_merged['is_holiday'].value_counts()

0    6258
1     148
Name: is_holiday, dtype: int64

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [28]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_merged, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['ID', 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['ID', 'Severity'])
y_val = val_df['Severity']

In [29]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_train, y_train)
sel.get_support()
X_train.columns[(sel.get_support())]

Index(['Lat', 'Lng', 'Distance(mi)', 'Weather_Condition', 'Temperature(F)',
       'Humidity(%)', 'Wind_Speed(mph)'],
      dtype='object')

As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

In [30]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
X_train = X_train[['Lat', 'Lng', 'Distance(mi)', 'Temperature(F)',
       'Humidity(%)', 'Wind_Speed(mph)']]

X_val = X_val[['Lat', 'Lng', 'Distance(mi)', 'Temperature(F)',
       'Humidity(%)', 'Wind_Speed(mph)']]

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [31]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=25,random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [32]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.843213728549142


Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [33]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58


Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [34]:
test_df.drop(columns=['Bump','Roundabout'],inplace=True)
test_df['timestamp']=pd.to_datetime(test_df['timestamp'])
test_df['date_time']=pd.to_datetime(test_df['timestamp'].dt.strftime('%Y-%m-%d %H:00:00'))


test_df['Crossing'] = test_df['Crossing'].astype('int')
test_df['Give_Way'] = test_df['Give_Way'].astype('int')
test_df['Junction'] = test_df['Junction'].astype('int')
test_df['No_Exit'] = test_df['No_Exit'].astype('int')
test_df['Railway'] = test_df['Railway'].astype('int')
test_df['Stop'] = test_df['Stop'].astype('int')
test_df['Amenity'] = test_df['Amenity'].astype('int')

test_df['Side'] = labelencoder.fit_transform(test_df['Side'])

test_df['is_holiday']=test_df['timestamp'].dt.date.isin(holidays_df['date'].dt.date).astype(int)  

test_df_merged =pd.merge(test_df, weather_df, how='left', on=['date_time'])

test_df_merged.dropna(inplace=True)

test_df_merged.drop(columns=['date_time','timestamp'],inplace=True)
test_df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1601 entries, 0 to 1600
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ID                 1601 non-null   int64  
 1   Lat                1601 non-null   float64
 2   Lng                1601 non-null   float64
 3   Distance(mi)       1601 non-null   float64
 4   Crossing           1601 non-null   int64  
 5   Give_Way           1601 non-null   int64  
 6   Junction           1601 non-null   int64  
 7   No_Exit            1601 non-null   int64  
 8   Railway            1601 non-null   int64  
 9   Stop               1601 non-null   int64  
 10  Amenity            1601 non-null   int64  
 11  Side               1601 non-null   int64  
 12  is_holiday         1601 non-null   int64  
 13  Weather_Condition  1601 non-null   int64  
 14  Temperature(F)     1601 non-null   float64
 15  Humidity(%)        1601 non-null   float64
 16  Wind_Speed(mph)    1601 

In [35]:
test_df_merged.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,is_holiday,Weather_Condition,Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi)
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,0,1,0,17,63.0,60.0,10.4,10.0
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,0,1,0,3,65.0,56.0,5.0,9.0
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,0,1,0,14,58.0,90.0,18.0,10.0
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,0,1,0,3,72.0,59.0,17.0,10.0
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,0,1,0,22,57.0,77.0,5.8,10.0


In [36]:
X_test = test_df_merged.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
X_test = X_test[['Lat', 'Lng', 'Distance(mi)', 'Temperature(F)',
       'Humidity(%)', 'Wind_Speed(mph)']]

y_test_predicted = classifier.predict(X_test)

test_df['Severity'] = y_test_predicted

test_df.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,timestamp,date_time,is_holiday,Severity
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,0,1,2016-04-04 19:20:31,2016-04-04 19:00:00,0,2
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,0,1,2020-10-28 11:51:00,2020-10-28 11:00:00,0,2
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,0,1,2019-09-09 07:36:45,2019-09-09 07:00:00,0,2
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,0,1,2019-08-06 15:46:25,2019-08-06 15:00:00,0,2
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,0,1,2018-10-17 09:54:58,2018-10-17 09:00:00,0,2


Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [37]:
test_df[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.